In [1]:
# Open one image
import numpy as np
archive = np.load("Chal/train_data/images/patient_002.npz")
scan = archive['scan']
mask = archive['mask']

In [9]:
# Convert to SimpleITK format data
import SimpleITK as sitk
scanimg = sitk.GetImageFromArray(scan)
sitk.WriteImage(scanimg, "Chal/train_data/images/patient_002_sikt_scan.nrrd") 
scanmask = sitk.GetImageFromArray(np.array(mask,dtype=int))
sitk.WriteImage(scanmask, "Chal/train_data/images/patient_002_sikt_mask.nrrd") 

In [29]:
# Show one form of tumor in 2D
from PIL import Image
Image.fromarray(mask[:,:,50]).show()

In [33]:
# Convert to Simple ITK format all image data
import os 
filenames = os.listdir("Chal/train_data/images")
for file in filenames:
    if file.split(".")[-1]!="npz":
        continue
    archive = np.load("Chal/train_data/images/"+file)
    scan = archive['scan']
    mask = archive['mask']
    scanimg = sitk.GetImageFromArray(scan)
    sitk.WriteImage(scanimg, "Chal/train_data/images/"+file.split(".")[0]+"_sikt_scan.nrrd") 
    scanmask = sitk.GetImageFromArray(np.array(mask,dtype=int))
    sitk.WriteImage(scanmask, "Chal/train_data/images/"+file.split(".")[0]+"_sikt_mask.nrrd")
    # np.save("Chal/train_data/images/"+file.split(".")[0]+"_scan.npy", scan)
    # np.save("Chal/train_data/images/"+file.split(".")[0]+"_mask.npy", mask)

In [96]:
# Convert to Simple ITK format all image data
import os 
filenames = os.listdir("Chal/test_data/images")
for file in filenames:
    if file.split(".")[-1]!="npz":
        continue
    archive = np.load("Chal/test_data/images/"+file)
    scan = archive['scan']
    mask = archive['mask']
    scanimg = sitk.GetImageFromArray(scan)
    sitk.WriteImage(scanimg, "Chal/test_data/images/"+file.split(".")[0]+"_sikt_scan.nrrd") 
    scanmask = sitk.GetImageFromArray(np.array(mask,dtype=int))
    sitk.WriteImage(scanmask, "Chal/test_data/images/"+file.split(".")[0]+"_sikt_mask.nrrd")
    # np.save("Chal/train_data/images/"+file.split(".")[0]+"_scan.npy", scan)
    # np.save("Chal/train_data/images/"+file.split(".")[0]+"_mask.npy", mask)

In [43]:
# Example of image without tumor
archive = np.load("Chal/train_data/images/patient_003.npz")
scan = archive['scan']
mask = archive['mask']
len(np.unique(mask))

1

In [11]:
# Radiomics Feature Extractor
from radiomics import featureextractor
extractor = featureextractor.RadiomicsFeatureExtractor()
print(extractor.featureClassNames)

['firstorder', 'glcm', 'gldm', 'glrlm', 'glszm', 'ngtdm', 'shape', 'shape2D']


In [36]:
extractor.__dict__

{'settings': {'minimumROIDimensions': 2,
  'minimumROISize': None,
  'normalize': False,
  'normalizeScale': 1,
  'removeOutliers': None,
  'resampledPixelSpacing': None,
  'interpolator': 'sitkBSpline',
  'preCrop': False,
  'padDistance': 5,
  'distances': [1],
  'force2D': False,
  'force2Ddimension': 0,
  'resegmentRange': None,
  'label': 1,
  'additionalInfo': True},
 'enabledImagetypes': {'Original': {}},
 'enabledFeatures': {'firstorder': [],
  'glcm': [],
  'gldm': [],
  'glrlm': [],
  'glszm': [],
  'ngtdm': [],
  'shape': []},
 'featureClassNames': ['firstorder',
  'glcm',
  'gldm',
  'glrlm',
  'glszm',
  'ngtdm',
  'shape',
  'shape2D']}

In [21]:
# Execute on one image
result = extractor.execute("Chal/train_data/images/patient_002_sikt_scan.nrrd", "Chal/train_data/images/patient_002_sikt_mask.nrrd")

for key, value in result.items():
    print(key, value)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


diagnostics_Versions_PyRadiomics 2.2.0
diagnostics_Versions_Numpy 1.17.4
diagnostics_Versions_SimpleITK 1.2.4
diagnostics_Versions_PyWavelet 0.5.2
diagnostics_Versions_Python 3.6.5
diagnostics_Configuration_Settings {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
diagnostics_Configuration_EnabledImageTypes {'Original': {}}
diagnostics_Image-original_Hash 12594f333e663f725eafbb011efb36cc28ef09ea
diagnostics_Image-original_Dimensionality 3D
diagnostics_Image-original_Spacing (1.0, 1.0, 1.0)
diagnostics_Image-original_Size (92, 92, 92)
diagnostics_Image-original_Mean -137.40974048656201
diagnostics_Image-original_Minimum -1023.0
diagnostics_Image-original_Maximum 1322.0
diagnostics_Mask-original_Hash acd42c9

In [85]:
# Extract features on all images
filenames = os.listdir("Chal/train_data/images")

features_names = list(result.keys())+["patient_number"]
features_list = dict.fromkeys(features_names)
for feature in features_names:
    features_list[feature] = []

for file in filenames:
    if file.split(".")[-1]=="npz":
        features_list["patient_number"].append(file.split(".")[-2].split("_")[1])

for file in filenames:
    if file.split(".")[-1]!="npz":
        continue
    """if int(file.split("_")[1].split(".")[0])==395:
        for feature in features_names:
            features_list[feature].append(None)
        continue"""
    scan_path = "Chal/train_data/images/"+file.split(".")[0]+"_sikt_scan.nrrd"
    mask_path = "Chal/train_data/images/"+file.split(".")[0]+"_sikt_mask.nrrd"
    if len(np.unique(sitk.GetArrayFromImage(sitk.ReadImage(mask_path))))<2:
        for feature in features_names:
            if feature!="patient_number":
                features_list[feature].append(None)
        continue
    print(file)
    result = extractor.execute(scan_path, mask_path)
    for key, value in result.items():
        features_list[key].append(value)

patient_002.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_004.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_005.npz
patient_007.npz


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_423.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_424.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [97]:
# Extract features on all images
filenames = os.listdir("Chal/test_data/images")

features_names = list(result.keys())+["patient_number"]
features_list = dict.fromkeys(features_names)
for feature in features_names:
    features_list[feature] = []

for file in filenames:
    if file.split(".")[-1]=="npz":
        features_list["patient_number"].append(file.split(".")[-2].split("_")[1])

for file in filenames:
    if file.split(".")[-1]!="npz":
        continue
    scan_path = "Chal/test_data/images/"+file.split(".")[0]+"_sikt_scan.nrrd"
    mask_path = "Chal/test_data/images/"+file.split(".")[0]+"_sikt_mask.nrrd"
    if len(np.unique(sitk.GetArrayFromImage(sitk.ReadImage(mask_path))))<2:
        for feature in features_names:
            if feature!="patient_number":
                features_list[feature].append(None)
        continue
    print(file)
    result = extractor.execute(scan_path, mask_path)
    for key, value in result.items():
        features_list[key].append(value)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_000.npz
patient_001.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_006.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...

patient_412.npz
patient_413.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_415.npz


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


patient_419.npz


In [50]:
# Useless
rows = []
for i in features_names:
    while "diagnostics"
rows.append(features_names)
for key, value in features_list.items():
    rows.append(value)

In [92]:
# Save Features
df = pd.DataFrame(data=features_list)
df.to_csv("extracted_features.csv", sep=',',index=False)

In [98]:
# Save Features
df = pd.DataFrame(data=features_list)
df.to_csv("test_extracted_features.csv", sep=',',index=False)

In [93]:
df

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,patient_number
0,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},12594f333e663f725eafbb011efb36cc28ef09ea,3D,"(1.0, 1.0, 1.0)",...,0.0014159294924583742,7.440152780881469,0.05065013935430697,811080.0103191268,4.638003576181803,5.446082904968082e-05,1208.990130928983,0.004242342429635777,0.3528600071678195,002
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,003
2,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},793be327098729932754ab0c539a10d3be8007c0,3D,"(1.0, 1.0, 1.0)",...,0.0011823588904757247,8.05320738907593,0.12583400659035412,43542.93247932123,1.2453260036682692,0.000216962582959674,2124.6732991252165,0.029745506287594543,0.593370980923771,004
3,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},5728d2537b555bb664252698dc1a51694866b4f4,3D,"(1.0, 1.0, 1.0)",...,0.006010770464847511,6.445624212509823,0.03442424242424243,78052.2017952787,0.3192361746468855,0.0011584858008878434,493.9439694152586,0.004182293302986566,7.9263924028377275,005
4,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9f1b576cfc98ab355c1ac62a08abd4f84c89be21,3D,"(1.0, 1.0, 1.0)",...,0.0010591164492006718,7.968079701923623,0.05924262819164598,611132.025210262,3.6014025715567675,5.676665846302051e-05,1723.9199416906906,0.009181368148031591,0.5111257271308385,007
5,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},6d505a43db6064df0112884d2876f6e83e97804c,3D,"(1.0, 1.0, 1.0)",...,0.002603921296331485,6.902112459727654,0.3567645642753972,297.2954117880604,0.29144824075065945,0.002476095943858015,1625.667447850248,0.23602064618873433,2.0443356629047984,008
6,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e73b20e78dcb299acfe3ccdb16915e5da5e64e8b,3D,"(1.0, 1.0, 1.0)",...,0.006495320016546854,5.951022804375938,0.36528028933092227,156.28264385844525,0.09501105398887026,0.007916250096069312,1518.545746076143,0.18237875682701976,5.4164580535629065,011
7,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},51a099ec121e376fb2ac8cd6c124e5022cb3a8e8,3D,"(1.0, 1.0, 1.0)",...,0.0024816657630558495,7.187488219746435,0.22692146021275908,12552.147271933809,0.718215089001439,0.0005338474193486882,1647.829002575773,0.15172573955141086,0.9576135144010168,014
8,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},a006099e256744eedd9278354a398b33c134a07e,3D,"(1.0, 1.0, 1.0)",...,0.002710463401557397,7.163170755024512,0.06724826260535266,210087.6836985185,3.411742942537483,0.00014734084864731234,571.386299734884,0.019888263426672353,0.39281939193215354,015
9,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},767e22198388ac93a9db95e0dfd9b3fa7e14f685,3D,"(1.0, 1.0, 1.0)",...,0.0011997397769629755,7.300068195325571,0.02935379145087733,686026.4747536255,3.568586114684688,9.763512851495345e-05,684.5476765671239,0.004234913657332878,0.5179838137907027,016


In [99]:
df

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,patient_number
0,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},a14c1903ca44b9c8bee21607abe86b6092e0e784,3D,"(1.0, 1.0, 1.0)",...,0.00650396833685046,7.030494272641832,0.3864306784660767,88.6209195268341,0.3192207510507349,0.0029973859057788035,2400.7459995508075,0.2587860762411749,3.0193578055002153,000
1,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},012bee7c6eb8bc6eb005674e2786b955b3f2d343,3D,"(1.0, 1.0, 1.0)",...,0.0011372251755402353,7.056755551475125,0.16735686583842765,28829.700225615623,1.0432795311296523,0.000299498303363116,1503.157610147304,0.05499763926588297,0.8124728927622762,001
2,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},57504b176455455b107b42a27734198353b891b5,3D,"(1.0, 1.0, 1.0)",...,0.0030927233376598366,7.295787467156062,0.16808601234830742,9195.18109360467,0.4240922951318243,0.0009696911689902665,1549.136212300552,0.04406541279692153,1.8312712830615954,006
3,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},466571f330c0e12c21c0e6b2ee64953b043a074a,3D,"(1.0, 1.0, 1.0)",...,0.011458708772295158,7.0290292656028655,0.466265441875198,11.03069203021621,3.3630953665269554,0.00046453293179421864,5166.658333866696,0.2505568367343866,0.7531802976962354,009
4,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e85f367b7409c24d3dda412f0616f26da1ea8d55,3D,"(1.0, 1.0, 1.0)",...,0.001263275408950132,7.296961213716396,0.11617878762749813,64430.20175176771,0.8221696618971971,0.00016990275935880697,5325.885346788066,0.010699961481524708,2.6618141092070435,010
5,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ad9134c02294af33a76dd455adb2ea656e73d0bb,3D,"(1.0, 1.0, 1.0)",...,0.0032969046158858,6.8678024800660165,0.34163816967920535,544.6214691280725,0.2863143301056496,0.0025691941507084907,1379.2028413909507,0.2628539609310901,2.0394846613420556,012
6,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d37a13f547ac65b0c300c7a8bfee84b3115e7c8c,3D,"(1.0, 1.0, 1.0)",...,0.0029809603474066533,7.738243027132657,0.03732633079136861,1309263.1233864485,7.805748110033149,4.954695416944482e-05,822.2480054589579,0.004100770738762614,0.2195195589714611,013
7,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},9f427e296f28dcda6c8ba2256469bf910fc1fdf0,3D,"(1.0, 1.0, 1.0)",...,0.00517823384912288,6.614141307851755,0.3871290430143381,45.285296393483,0.2705994297708851,0.004530731014635755,1220.8504664217726,0.20120530288705538,2.1775998993460277,019
8,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},65e056af2c7644355d20656e3143dd6ec4212f41,3D,"(1.0, 1.0, 1.0)",...,0.0010366632480026004,7.11280310139011,0.0696949112495939,605556.5375066835,4.077923636426653,6.20967645037254e-05,1006.0517011238957,0.010350257260902742,0.4301401259562533,027
9,2.2.0,1.17.4,1.2.4,0.5.2,3.6.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},f59894d720e8d39348e9565d038b4d67f5b75b73,3D,"(1.0, 1.0, 1.0)",...,0.010479469010200998,7.093423309251276,0.5298420976387078,43.43556805391967,0.8254093902613867,0.00069

In [102]:
list(features_list.keys())

['diagnostics_Versions_PyRadiomics',
 'diagnostics_Versions_Numpy',
 'diagnostics_Versions_SimpleITK',
 'diagnostics_Versions_PyWavelet',
 'diagnostics_Versions_Python',
 'diagnostics_Configuration_Settings',
 'diagnostics_Configuration_EnabledImageTypes',
 'diagnostics_Image-original_Hash',
 'diagnostics_Image-original_Dimensionality',
 'diagnostics_Image-original_Spacing',
 'diagnostics_Image-original_Size',
 'diagnostics_Image-original_Mean',
 'diagnostics_Image-original_Minimum',
 'diagnostics_Image-original_Maximum',
 'diagnostics_Mask-original_Hash',
 'diagnostics_Mask-original_Spacing',
 'diagnostics_Mask-original_Size',
 'diagnostics_Mask-original_BoundingBox',
 'diagnostics_Mask-original_VoxelNum',
 'diagnostics_Mask-original_VolumeNum',
 'diagnostics_Mask-original_CenterOfMassIndex',
 'diagnostics_Mask-original_CenterOfMass',
 'original_shape_Elongation',
 'original_shape_Flatness',
 'original_shape_LeastAxisLength',
 'original_shape_MajorAxisLength',
 'original_shape_Maximum